Pierre Navaro - [Institut de Recherche Mathématique de Rennes](https://irmar.univ-rennes1.fr) - [CNRS](http://www.cnrs.fr/)

In [1]:
%env CC='gcc-7'
%env CXX='g++-7'

env: CC='gcc-7'
env: CXX='g++-7'


In [2]:
import numpy as np

# Wrapping Mersenne twister C++ class in Cython

Example MT_RNG Class from [K. W. Smith Book examples](https://github.com/cythonbook/examples)


In [3]:
%rm -rf examples
!git clone https://github.com/cythonbook/examples.git

Cloning into 'examples'...
remote: Counting objects: 1737, done.
remote: Total 1737 (delta 0), reused 0 (delta 0), pack-reused 1737
Receiving objects: 100% (1737/1737), 934.05 KiB | 848.00 KiB/s, done.
Resolving deltas: 100% (827/827), done.


In [4]:
%cd /Users/navaro/osur-python-2017/examples/08-wrapping-cxx/01-simple-example-mt_rng-class

/Users/navaro/osur-python-2017/examples/08-wrapping-cxx/01-simple-example-mt_rng-class


A random generator is implemented in a C++ class with the following interface:

```C++
namespace mtrandom {

const static unsigned int N = 624;

class MT_RNG
{
  public:
    MT_RNG();
    MT_RNG(unsigned long s);
    MT_RNG(unsigned long init_key[], int key_length);

    // initializes RNG state, called by constructors.
    void init_genrand(unsigned long s);

    /* generates a random number on [0,0xffffffff]-interval */
    unsigned long genrand_int32();

    /* generates a random number on [0,0x7fffffff]-interval */
    long genrand_int31();

    /* generates a random number on [0,1]-real-interval */
    double genrand_real1();

    /* generates a random number on [0,1)-real-interval */
    double genrand_real2();

    /* generates a random number on (0,1)-real-interval */
    double genrand_real3();

    /* generates a random number on [0,1) with 53-bit resolution*/
    double genrand_res53();

    double operator()() {
      return genrand_real1();
    }

  private:

    unsigned long mt[N];
    int mti;
}; // class MT_RNG

} // namespace mtrandom
```

* Cython can only wrap public methods and members; any private or protected methods or members are not accessible, and thus not wrappable.
* To declare this class interface for use in Cython, we use an extern block as before. This extern block requires three additional elements to handle C++-isms:
    - Declaring the C++ namespace with the Cython namespace clause
    - Using the cppclass keyword to declare a C++ class interface block
    - Declaring the class’s interface in this block

```cython
# distutils: language = c++
# distutils: sources = mt19937.cpp

cdef extern from "mt19937.h" namespace "mtrandom":
    unsigned int N
    cdef cppclass MT_RNG:
        MT_RNG()
        MT_RNG(unsigned long s)
        MT_RNG(unsigned long init_key[], int key_length)
        void init_genrand(unsigned long s)
        unsigned long genrand_int32()
        double genrand_real1()
        double operator()()

cdef class RNG: # Wrapper extension type has a pointer 
                # to a heap-allocated instance of the C++ 
                # class it is wrapping:

    cdef MT_RNG *_thisptr

    # Method to create and initialize a valid MT_RNG object.
    def __cinit__(self, unsigned long s):
        self._thisptr = new MT_RNG(s)
        if self._thisptr == NULL:
            raise MemoryError()
    # Method for finalization.
    def __dealloc__(self):
        if self._thisptr != NULL:
            del self._thisptr

    # cpdef methods to generate random numbers from Python
    cpdef unsigned long randint(self):
        return self._thisptr.genrand_int32()

    cpdef double rand(self):
        return self._thisptr.genrand_real1()
```

# Compiling with C++

With compiler directives inside RNG.pyx, the
distutils script is simpler. 
```C
# distutils: language = c++
# distutils: sources = mt19937.cpp
```

In [5]:
%%file setup.py
from distutils.core import setup, Extension
from Cython.Build import cythonize
import os
os.environ["CC"] = 'gcc-7'
os.environ["CXX"] = 'g++-7'

ext = Extension("RNG",
                sources=["RNG.pyx", "mt19937.cpp"],
                language="c++")

setup(name="RNG",
      ext_modules=cythonize(ext))

Overwriting setup.py


In [7]:
!python3 setup.py build_ext -if

WARNING  Pythran support disabled for module: omp
Compiling RNG.pyx because it changed.
[1/1] Cythonizing RNG.pyx
running build_ext
building 'RNG' extension
C compiler: gcc-7 -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes

creating build
creating build/temp.macosx-10.12-x86_64-3.6
compile options: '-I/usr/local/include -I/usr/local/opt/openssl/include -I/usr/local/opt/sqlite/include -I/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/include/python3.6m -c'
gcc-7: RNG.cpp
cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++
gcc-7: mt19937.cpp
cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++
mt19937.cpp: In member function 'void mtrandom::MT_RNG::init_genrand(long unsigned int)':
mt19937.cpp:23:18: warning: comparison between signed and unsigned integer expressions [-Wsign-compare]
   for (mti=1; mti<

## Using the Wrapper from Python

In [8]:
from RNG import RNG
r = RNG(42)

In [9]:
r.randint(), r.randint()

(1608637542, 3421126067)

In [10]:
r.rand(), r.rand()

(0.9507143117838339, 0.1834347877147223)

## Overloaded Methods and Functions

The MT_RNG class has an alternate constructor that takes an array of unsigned longs to initialize the random-number generator’s state. How can we call this from Python?

We can do this by checking argument types of `__cinit__` method. We use an cython array which has a similar interface to a list, but it requires that all contained elements have the same scalar C type. Modify the RNG.pyx file:
```cython
from cpython.array cimport array
# ...
    def __cinit__(self, seed_or_state): 
        cdef array state_arr
        if isinstance(seed_or_state, int):
            self._thisptr = new MT_RNG(seed_or_state) 
        else:
            state_arr = array("L", seed_or_state)
            self._thisptr = new MT_RNG(state_arr.data.as_ulongs,
                                       len(state_arr))
```
If a list is passed to the constructor, it is converted to an array and passed to the second constructor.

*NumPy array could be used but it is less simple and introduces an external dependency.*

In [11]:
%%file RNG.pyx
# distutils: language = c++
# distutils: sources = mt19937.cpp
from cpython.array cimport array

cdef extern from "mt19937.h" namespace "mtrandom":
    unsigned int N
    cdef cppclass MT_RNG:
        MT_RNG()
        MT_RNG(unsigned long s)
        MT_RNG(unsigned long init_key[], int key_length)
        void init_genrand(unsigned long s)
        unsigned long genrand_int32()
        double genrand_real1()
        double operator()()

cdef class RNG:

    cdef MT_RNG *_thisptr

    def __cinit__(self, seed_or_state): 
        
        cdef array state_arr
        if isinstance(seed_or_state, int):
            self._thisptr = new MT_RNG(seed_or_state) 
        else:
            state_arr = array("L", seed_or_state)
            self._thisptr = new MT_RNG(state_arr.data.as_ulongs,
                                       len(state_arr))
        if self._thisptr == NULL:
            raise MemoryError()

    def __dealloc__(self):
        if self._thisptr != NULL:
            del self._thisptr

    cpdef unsigned long randint(self):
        return self._thisptr.genrand_int32()

    cpdef double rand(self):
        return self._thisptr.genrand_real1()


Overwriting RNG.pyx


In [12]:
!make clean
!python3 setup.py build_ext -if

rm -r RNG.cpp RNG.so build
rm: RNG.so: No such file or directory
make: [clean] Error 1 (ignored)
WARNING  Pythran support disabled for module: omp
Compiling RNG.pyx because it changed.
[1/1] Cythonizing RNG.pyx
running build_ext
building 'RNG' extension
C compiler: gcc-7 -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes

creating build
creating build/temp.macosx-10.12-x86_64-3.6
compile options: '-I/usr/local/include -I/usr/local/opt/openssl/include -I/usr/local/opt/sqlite/include -I/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/include/python3.6m -c'
gcc-7: RNG.cpp
cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++
gcc-7: mt19937.cpp
cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++
mt19937.cpp: In member function 'void mtrandom::MT_RNG::init_genrand(long unsigned int)':
mt19937.cpp:23:18: warning

### Using the Wrapper from Python

In [ ]:
import os
os._exit(00)

In [4]:
%cd /Users/navaro/osur-python-2017/examples/08-wrapping-cxx/01-simple-example-mt_rng-class
from RNG import RNG
r2 = RNG(list(range(30,40)))

/Users/navaro/osur-python-2017/examples/08-wrapping-cxx/01-simple-example-mt_rng-class


In [5]:
r2.rand()

0.04691027990703245

In [6]:
r2.randint()

2626217183

# Operator Overloading
- Cython supports most C++ operator overloads. 
- Currently, the in-place operators (+=, -=, etc.) are not supported. 
- Some operators are incompatible with Python’s syntax, so Cython provides a special [cython.operators](http://cython.readthedocs.io/en/latest/src/userguide/wrapping_CPlusPlus.html#c-operators-not-compatible-with-python-syntax) magic module to allow Python-compatible access. 

In [7]:
%%file RNG.pyx
# distutils: language = c++
# distutils: sources = mt19937.cpp
from cpython.array cimport array
from cython.operator cimport dereference as deref # new import

cdef extern from "mt19937.h" namespace "mtrandom":
    unsigned int N
    cdef cppclass MT_RNG:
        MT_RNG()
        MT_RNG(unsigned long s)
        MT_RNG(unsigned long init_key[], int key_length)
        void init_genrand(unsigned long s)
        unsigned long genrand_int32()
        double genrand_real1()
        double operator()()

cdef class RNG:

    cdef MT_RNG *_thisptr

    def __cinit__(self, seed_or_state): 
        
        cdef array state_arr
        if isinstance(seed_or_state, int):
            self._thisptr = new MT_RNG(seed_or_state) 
        else:
            state_arr = array("L", seed_or_state)
            self._thisptr = new MT_RNG(state_arr.data.as_ulongs,
                                       len(state_arr))
        if self._thisptr == NULL:
            raise MemoryError()
        
    def __call__(self):               # overload () operator
        return deref(self._thisptr)()

    def __dealloc__(self):
        if self._thisptr != NULL:
            del self._thisptr

    cpdef unsigned long randint(self):
        return self._thisptr.genrand_int32()

    cpdef double rand(self):
        return self._thisptr.genrand_real1()

Overwriting RNG.pyx


In [8]:
!make clean
!python3 setup.py build_ext -if

rm -r RNG.cpp RNG.so build
rm: RNG.so: No such file or directory
make: [clean] Error 1 (ignored)
WARNING  Pythran support disabled for module: omp
Compiling RNG.pyx because it changed.
[1/1] Cythonizing RNG.pyx
running build_ext
building 'RNG' extension
C compiler: gcc-7 -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes

creating build
creating build/temp.macosx-10.12-x86_64-3.6
compile options: '-I/usr/local/include -I/usr/local/opt/openssl/include -I/usr/local/opt/sqlite/include -I/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/include/python3.6m -c'
gcc-7: RNG.cpp
cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++
gcc-7: mt19937.cpp
cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++
mt19937.cpp: In member function 'void mtrandom::MT_RNG::init_genrand(long unsigned int)':
mt19937.cpp:23:18: warning

### Use the new operator

In [ ]:
import os
os._exit(00)

In [1]:
%cd /Users/navaro/osur-python-2017/examples/08-wrapping-cxx/01-simple-example-mt_rng-class
from RNG import RNG

/Users/navaro/osur-python-2017/examples/08-wrapping-cxx/01-simple-example-mt_rng-class


In [2]:
r = RNG(10)
r()

0.7713206433158649

In [3]:
[r() for i in range(3)]

[0.29876115855266366, 0.02075194661057367, 0.49458992841993227]

# Packages - pxd files

Cython uses .pxd files which work like C header files – they contain Cython declarations (and sometimes code sections) which are only meant for inclusion by Cython modules. A pxd file is imported into a pyx module by using the cimport keyword.

1. They can be used for sharing external C declarations.
2. They can contain functions which are well suited for inlining by the C compiler. Such functions should be marked inline, example:
```cython
cdef inline int int_min(int a, int b):
    return b if b < a else a
```
3. When accompanying an equally named pyx file, they provide a Cython interface to the Cython module so that other Cython modules can communicate with it using a more efficient protocol than the Python one.

http://cython.readthedocs.io/en/latest/src/tutorial/pxd_files.html